# 21_6 Descriptive Analysis - Lines, Types, Service

Let us now take a closer look on the characterisitcs of our aggregrated data.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib
import holidays
from shapely import Point, wkt
import scipy.stats
import seaborn as sns

## Load and prepare data

In [2]:
data_month_double = '07'

In [3]:
# read final data
data = pd.read_csv("../Data/20_Final_Data.csv")

/tmp/ipykernel_384496/974816303.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../../../20_Final_Data_NEW_TEST.csv")


In [4]:
tier = pd.read_parquet("../Data/12_tier_part2.parquet")
next = pd.read_parquet("../Data/15_nextbike_part2.parquet")

In [5]:
tier['trip_duration_seconds'] = tier['trip_duration'].dt.total_seconds()
next['trip_duration_seconds'] = next['trip_duration'].dt.total_seconds()

In [6]:
# transform geometry
data['station_point'] = data['station_point'].apply(wkt.loads)

In [7]:
data['buffer_zone'] = shapely.wkt.loads(data['buffer_zone'])

In [8]:
data = gpd.GeoDataFrame(data, geometry='station_point')

In [9]:
data = data.loc[:, ~data.columns.str.contains('Unnamed')]

In [10]:
#data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'], inplace=True)

In [11]:
data['actual_arrival_time'] = pd.to_datetime(data['actual_arrival_time'])
data['actual_departure_time'] = pd.to_datetime(data['actual_departure_time'])
data['scheduled_arrival_time'] = pd.to_datetime(data['scheduled_arrival_time'])
data['scheduled_departure_time'] = pd.to_datetime(data['scheduled_departure_time'])

In [12]:
data = gpd.GeoDataFrame(data, geometry='buffer_zone')

In [13]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2564861 entries, 0 to 2564860
Data columns (total 56 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   route_id                             int64         
 1   agency_id                            int64         
 2   route_short_name                     object        
 3   route_type                           int64         
 4   route_type_name                      object        
 5   agency_name                          object        
 6   service_id                           int64         
 7   trip_id                              object        
 8   trip_headsign                        object        
 9   direction_id                         int64         
 10  shape_id                             int64         
 11  stop_id                              int64         
 12  actual_arrival_time                  datetime64[ns]
 13  actual_departure_ti

In [14]:
data['date'] = pd.to_datetime(data['date'])

In [15]:
data['arrival_delay'] = pd.to_timedelta(data['arrival_delay'])
data['departure_delay'] = pd.to_timedelta(data['departure_delay'])

In [16]:
data['arrival_delay_float'] = data['arrival_delay'].astype(str)
data['arrival_delay_float'] = data['arrival_delay_float'].str.split(' ', n=2).str[-1]
data['arrival_delay_float'] = data['arrival_delay_float'].astype(str)
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float'])
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float']).dt.total_seconds() / 60

In [17]:
data['departure_delay_float'] = data['departure_delay'].astype(str)
data['departure_delay_float'] = data['departure_delay_float'].str.split(' ', n=2).str[-1]
data['departure_delay_float'] = data['departure_delay_float'].astype(str)
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float'])
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float']).dt.total_seconds() / 60

In [18]:
#for index in data.index:   
 #   if data['scheduled_departure_time'][index] < data['scheduled_arrival_time'][index]:
  #      if (data['actual_arrival_time'][index] is not pd.NaT):
   #         data['arrival_delay'][index] = data['actual_arrival_time'][index] - data['scheduled_arrival_delay'][index]

In [19]:
#NEW FEATURES
data['delay_category'] = data['departure_delay'].apply(lambda x: 1 if x > pd.Timedelta(0) else 0 if x == pd.Timedelta(0) else -1)
data['cancelled_trip'] = np.where((data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()), 1, 0)

In [20]:
# adjust delay category for trips that started
for index in data.index:   
    if pd.isnull(data['actual_arrival_time'][index]) & (data['actual_departure_time'][index] is not pd.NaT):
        if data['departure_delay'][index] == pd.Timedelta(0):
            data['delay_category'][index] = 0
        elif data['departure_delay'][index] > pd.Timedelta(0):
            data['delay_category'][index] = 0

/tmp/ipykernel_384496/2565516100.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['delay_category'][index] = 0
/tmp/ipykernel_384496/2565516100.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

## Data Description

### Definition of functions

In [21]:
def calc_stats(data, column_name):

    #generated by Copilot

    sum = data[column_name].sum()

    rows = len(data.index)

#Central Tendency

    # Calculate mean (average)
    mean = data[column_name].mean()
    # Calculate median (middle value)
    median = data[column_name].median()
    # Calculate mode (most frequent value)
    mode = data[column_name].mode()
    #calc maximum
    max = data[column_name].max()
    #calc minimum
    min = data[column_name].min()

    # Calculate range
    data_range = data[column_name].max() - data[column_name].min()
    # Calculate coefficient of variation of range
    cvr = data_range / mean
    # Calculate coefficient of variation of mean
    cvm = mean / mean
    # Calculate coefficient of variation of median
    cvmed = median / mean
    # Calculate coefficient of variation of mode
    cvmode = mode / mean
    # Calculate coefficient of variation of standard deviation
   

# Variability
# Measure of how spread out the values in a data set are
    
    # Calculate standard deviation (average amount of variability in a dataset; tells how far each score lies from the mean; the larger the standard deviation, the more variable the data)
    std_dev = data[column_name].std()
    # Calculate variance (average squared deviation from the mean; square of standard deviation; units of varriance are much larger than those of a typical value in the dataset)
    variance = data[column_name].var()
    # Calculate coefficient of variation
    cv = std_dev / mean # standard deviation divided by the mean; measures the relative variability of a dataset; allows comparison of variability of datasets with different units of measurement
    # Calculate 25th percentile
    percentile_25 = data[column_name].quantile(0.25)
    # Calculate 75th percentile
    percentile_75 = data[column_name].quantile(0.75)
    # Calculate interquartile range
    iqr = percentile_75 - percentile_25 # difference between the 75th and 25th percentiles; measures the spread of the middle 50% of values in a dataset
    # Calculate coefficient of quartile deviation
    qd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of coefficient of variation
    cvcv = std_dev / mean
    # Calculate coefficient of variation of coefficient of quartile deviation
    cvqd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of range

#Measure of Shape

    # Calculate skewness - measure of how much the probability distribution of a random variable deviates from the normal distribution
    skewness = data[column_name].skew()
    
    # Calculate kurtosis
    kurtosis = data[column_name].kurtosis()

    # Calculate coefficient of skewness
    cs = skewness / std_dev
    # Calculate coefficient of kurtosis
    ck = kurtosis / std_dev
    # Calculate coefficient of variation of skewness
    cvs = skewness / std_dev
    # Calculate coefficient of variation of kurtosis
    cvk = kurtosis / std_dev
    # Calculate coefficient of variation of coefficient of skewness
    cvcs = skewness / std_dev
    # Calculate coefficient of variation of coefficient of kurtosis
    cvck = kurtosis / std_dev
    
    
    
    #return "column: " + str(column_name), "mean: " + str(mean), "median: " + str(median), "mode: " + str(mode), "std_dev: " + str(std_dev), "variance: " + str(variance), "skewness: "+ str(skewness), "kurtosis: " + str(kurtosis), "percentile_25: "+ str(percentile_25), "percentile_75: "+ str(percentile_75), "iqr: " + str(iqr), "range: "+ str(data_range), "cv"+ str(cv), "qd: "+str(qd), "cs: "+ str(cs), "ck: " + str(ck), "csv: "+ str(cvs), "cvk: "+ str(cvk), "cvcs: "+ str(cvcs), "cvck: " +str(cvck), "cvcv: "+ str(cvcv), "cvqd: "+ str (cvqd), "cvr: "+ str(cvr), "cvm: " + str(cvm), "cvmed: " + str(cvmed), "cvmode: "+ str(cvmode)

    return pd.DataFrame({'column': [column_name],
                            'rows': [rows],
                         'sum': [sum],
                         'mean': [mean],
                         'median': [median],
                         'mode': [mode],
                         'max': [max],
                         'min': [min],
                         'std_dev': [std_dev],
                         'variance': [variance],
                         'skewness': [skewness],
                         'kurtosis': [kurtosis],
                         'percentile_25': [percentile_25],
                         'percentile_75': [percentile_75],
                         'iqr': [iqr],
                         'range': [data_range],
                         'cv': [cv],
                         'qd': [qd],
                         'cs': [cs],
                         'ck': [ck],
                         'cvs': [cvs],
                         'cvk': [cvk],
                         'cvcs': [cvcs],
                         'cvck': [cvck],
                         'cvcv': [cvcv],
                         'cvqd': [cvqd],
                         'cvr': [cvr],
                         'cvm': [cvm],
                         'cvmed': [cvmed],
                         'cvmode': [cvmode]}).T

In [22]:
# create a shorter dataframe with most interesting columns - for better overview
def show_short_df(dataframe):
    delay_short = dataframe[['route_id', 'trip_id', 'stop_id', 'stop_name', 'scheduled_arrival_time', 'actual_arrival_time', 'scheduled_departure_time', 'actual_departure_time', 'arrival_delay', 'departure_delay', 'tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'buffer_zone', 'arrival_delay_float', 'departure_delay_float']]
    return delay_short

In [23]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["end_location"], crs=4326))
# drop geometry columns in wrong format
tier = tier.drop(columns=["start_location", "end_location"])
# replace geometry columns with correct format
tier['start_location'] = point_start
tier['end_location'] = point_end

In [24]:
# get the data from the tier / nextbike dataset
def check_micromobility_datasets(type, data):
    
    if type == 'tier':

        return_data = tier.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['tier_trips_id'].str.split(' ').explode().dropna():   # iterate over all tier ids in the dataframe
            
            id = tier[tier['tier_trips_id'] == int(float(item))]   # get the rows with the specific id
            return_data = pd.concat([return_data, id])  # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)
            
    if type == 'next':

        return_data = next.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['nextbike_trips_id'].str.split(' ').explode().dropna():   # iterate over all nextbike ids in the dataframe
            
            id = next[next['nextbike_trips_id'] == int(float(item))]  # get the rows with the specific id
            return_data = pd.concat([return_data, id]) # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)

    return return_data

In [25]:
def get_station_to_station(data):
    t = check_micromobility_datasets('tier', data)
    n = check_micromobility_datasets('nextbike', data)

    t = t[t['end_stop_id'].notna() & t['start_stop_id'].notna()]
    n = n[n['end_stop_id'].notna() & n['start_stop_id'].notna()]
    
    return t, n

In [26]:
# map the start and end locations of the trips on a map and connect them with a line

def map_buffer_zones(dataset):
    
    micromobility_data = check_micromobility_datasets('tier', dataset)
    
    # Create a folium map object
    m = folium.Map(location=[50.73743, 7.09821], zoom_start=12)

    # Iterate over each row in the micromobility data
    for index, row in micromobility_data.iterrows():
        # Get the start and end locations
        start_location = row['start_location']
        end_location = row['end_location']
        
        # Get the start and end stop names
        start_stop_name = row['start_stop_name']
        end_stop_name = row['end_stop_name']
        # Add markers for the start and end locations
        folium.Marker(location=[start_location.y, start_location.x], popup=start_stop_name).add_to(m)
        folium.Marker(location=[end_location.y, end_location.x], popup=end_stop_name).add_to(m)
        # Create a line connecting the start and end locations
        folium.PolyLine(locations=[[start_location.y, start_location.x], [end_location.y, end_location.x]], color='blue').add_to(m)
        
    # Display the map
    return m

In [27]:
def get_stats(data, var):
    
    df = pd.DataFrame()

    for i in var:
        df = pd.concat([df, calc_stats(data, i)], axis=1)

    df.columns = df.iloc[0]
    df = df[1:]
    
    return df

## Create Delay, No_Delay, and Nan-Delay Datasets

In [28]:
delay = data[data['departure_delay'] > pd.Timedelta(0)]


In [29]:
#no_delay = data[data['arrival_delay'] == pd.Timedelta(0)]
no_delay = data[
    (data['departure_delay'] == pd.Timedelta(0)) | 
    (data['scheduled_arrival_time'].notna() & 
     data['actual_arrival_time'].isna() & 
     data['scheduled_departure_time'].notna() & 
     data['actual_departure_time'].notna())
]

In [30]:
#nan_delay = data[data['arrival_delay'].isna()]
nan_delay = data[data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()]

As we can see, there was no holiday in June.

#### Lines, Route Types, and Agency

In [31]:
data['route_short_name'].unique()

array(['551', '16', '66', '857', 'SB55', '640', 'N7', 'N4', 'N2', 'N5',
       'N8', '855', '18', '537', '845', 'N6', '65', 'SB60', '612', '609',
       '610', '638', '607', '611', '608', '602', '604', '601', '613',
       '605', '639', '614', '606', '631', '603', '600', '633', '856',
       '843', '550', '636', '800', '884', '817', '632', '516', '630',
       '63', '635', '540', '637', 'N10', 'N3', 'N9', 'N1', '842', '541',
       '634', '68', 'BE62'], dtype=object)

In [32]:
line_delay_trips = delay.groupby('route_short_name').agg({'tier_trips_count': 'sum', 'nextbike_trips_count': 'sum', 'trip_id': 'count', 'departure_delay_float': 'mean'})
line_delay_trips = line_delay_trips.merge(delay[['route_type_name', 'route_short_name']],  on='route_short_name', how='left')
line_delay_trips.drop_duplicates(inplace=True)
line_delay_trips

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
0,18,0,0,3,1.000000,Tram
3,540,311,581,10082,0.462879,Bus
10085,550,10,17,256,0.406250,Bus
10341,551,21,12,679,0.000000,Bus
11020,600,693,2248,40648,0.526053,Bus
51668,601,930,3201,63222,0.555645,Bus
114890,602,825,3387,51374,0.599832,Bus
166264,603,921,2539,47846,0.556771,Bus
214110,604,805,1819,54091,0.817118,Bus
268201,605,766,1752,49153,0.888389,Bus


In [134]:
t = line_delay_trips.sort_values(by='tier_trips_count', ascending=False).head(20)

In [135]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
51668,601,930,3201,63222,0.555645,Bus
166264,603,921,2539,47846,0.556771,Bus
114890,602,825,3387,51374,0.599832,Bus
214110,604,805,1819,54091,0.817118,Bus
317354,606,802,1530,53874,0.567458,Bus
403396,608,776,1910,52315,0.535454,Bus
566778,611,772,2133,79636,0.580454,Bus
268201,605,766,1752,49153,0.888389,Bus
455711,609,746,1937,51015,0.549294,Bus
11020,600,693,2248,40648,0.526053,Bus


In [128]:
t.to_csv("Tables/line_delay_tier.csv")

In [132]:
t = line_delay_trips.sort_values(by='departure_delay_float', ascending=False).head(20)

In [133]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
833065,639,27,29,5243,9.159880,Bus
941485,N6,2,0,272,8.480699,Bus
941757,N7,7,4,1726,8.162514,Bus
939206,N5,7,3,2279,6.608161,Bus
931718,N10,8,8,1991,6.382764,Bus
945978,N9,6,5,1808,5.592920,Bus
935321,N4,28,6,3885,5.315187,Bus
930720,N1,4,4,998,4.302355,Bus
933709,N2,2,3,985,3.804822,Bus
818380,637,54,74,3709,3.631437,Bus


In [131]:
t.to_csv("Tables/line_delay_delay.csv")

In [138]:
t = line_delay_trips.sort_values(by='nextbike_trips_count', ascending=False).head(20)

In [139]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
114890,602,825,3387,51374,0.599832,Bus
51668,601,930,3201,63222,0.555645,Bus
166264,603,921,2539,47846,0.556771,Bus
11020,600,693,2248,40648,0.526053,Bus
566778,611,772,2133,79636,0.580454,Bus
455711,609,746,1937,51015,0.549294,Bus
403396,608,776,1910,52315,0.535454,Bus
214110,604,805,1819,54091,0.817118,Bus
506726,610,675,1793,60052,0.667713,Bus
268201,605,766,1752,49153,0.888389,Bus


In [140]:
t.to_csv("Tables/line_delay_next.csv")

In [120]:
line_no_delay_trips = no_delay.groupby('route_short_name').agg({'tier_trips_count': 'sum', 'nextbike_trips_count': 'sum', 'trip_id': 'count', 'departure_delay_float': 'mean'})
line_no_delay_trips = line_no_delay_trips.merge(delay[['route_type_name', 'route_short_name']],  on='route_short_name', how='left')
line_no_delay_trips.drop_duplicates(inplace=True)
line_no_delay_trips

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
0,16,82,189,3048,0.000000,NaN
1,18,22,24,467,0.000000,Tram
4,516,49,14,9800,0.000000,NaN
5,537,619,1183,24333,0.000000,NaN
6,540,391,1077,6702,0.062146,Bus
10088,541,16,24,3194,0.000000,NaN
10089,550,532,1704,10979,0.000228,Bus
10345,551,821,2011,15257,0.000000,Bus
11024,600,656,1399,18532,0.032336,Bus
51672,601,784,1587,21193,0.064868,Bus


In [141]:
t = line_no_delay_trips.sort_values(by='tier_trips_count', ascending=False).head(20)

In [142]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
903394,66,1545,2958,31516,0.009926,Tram
566782,611,1201,2151,35002,0.027841,Bus
506730,610,1070,1960,23762,0.066998,Bus
455715,609,934,1748,23284,0.028657,Bus
403400,608,904,1758,22172,0.029519,Bus
317358,606,853,2480,24895,0.048684,Bus
855864,65,851,1939,33063,0.024650,Tram
10345,551,821,2011,15257,0.000000,Bus
51672,601,784,1587,21193,0.064868,Bus
166268,603,772,1455,20062,0.053048,Bus


In [143]:
t.to_csv("Tables/line_no_delay_tier.csv")

In [144]:
t = line_no_delay_trips.sort_values(by='nextbike_trips_count', ascending=False).head(20)

In [145]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
903394,66,1545,2958,31516,0.009926,Tram
214114,604,697,2613,20582,0.072041,Bus
317358,606,853,2480,24895,0.048684,Bus
268205,605,708,2444,20114,0.075967,Bus
566782,611,1201,2151,35002,0.027841,Bus
10345,551,821,2011,15257,0.000000,Bus
506730,610,1070,1960,23762,0.066998,Bus
855864,65,851,1939,33063,0.024650,Tram
403400,608,904,1758,22172,0.029519,Bus
455715,609,934,1748,23284,0.028657,Bus


In [146]:
t.to_csv("Tables/line_no_delay_next.csv")

In [147]:
t = line_no_delay_trips.sort_values(by='departure_delay_float', ascending=False).head(20)

In [148]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
941498,N6,2,1,155,2.248387,Bus
833069,639,10,11,2083,1.568771,Bus
941770,N7,4,6,698,0.654370,Bus
818384,637,7,4,1041,0.619116,Bus
939219,N5,2,2,332,0.566265,Bus
933722,N2,0,1,168,0.513393,Bus
802799,635,19,18,2635,0.287476,Bus
930733,N1,7,5,301,0.224252,Bus
934707,N3,2,3,223,0.213004,Bus
945991,N9,1,5,240,0.195833,Bus


In [149]:
t.to_csv("Tables/line_no_delay_delay.csv")

In [33]:
line_nan_trips = nan_delay.groupby('route_short_name').agg({'tier_trips_count': 'sum', 'nextbike_trips_count': 'sum', 'trip_id': 'count', 'departure_delay_float': 'mean'})
line_nan_trips = line_nan_trips.merge(nan_delay[['route_type_name', 'route_short_name']],  on='route_short_name', how='left')
line_nan_trips.drop_duplicates(inplace=True)
line_nan_trips

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
0,16,57,143,1900,NaN,Tram
1900,18,31,60,306,NaN,Tram
2206,516,56,35,12079,NaN,Bus
14285,537,624,1447,19316,NaN,Bus
33601,540,507,1234,12685,NaN,Bus
46286,541,9,21,3266,NaN,Bus
49552,550,197,667,4579,NaN,Bus
54131,551,658,1998,9647,NaN,Bus
63778,600,856,2386,38830,NaN,Bus
102608,601,771,2498,46788,NaN,Bus


In [150]:
t = line_nan_trips.sort_values(by='tier_trips_count', ascending=False).head(20)

In [151]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
911535,66,1473,3774,42603,NaN,Tram
855248,65,1089,2536,56287,NaN,Tram
392575,608,1061,2142,57517,NaN,Bus
279246,605,944,2638,45727,NaN,Bus
450092,609,927,1964,56252,NaN,Bus
149396,602,885,2700,45530,NaN,Bus
324973,606,882,2100,41114,NaN,Bus
63778,600,856,2386,38830,NaN,Bus
506344,610,853,1884,48897,NaN,Bus
234872,604,824,2473,44374,NaN,Bus


In [152]:
t.to_csv("Tables/line_nan_delay_tier.csv")

In [153]:
t = line_nan_trips.sort_values(by='nextbike_trips_count', ascending=False).head(20)

In [154]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
911535,66,1473,3774,42603,NaN,Tram
149396,602,885,2700,45530,NaN,Bus
279246,605,944,2638,45727,NaN,Bus
855248,65,1089,2536,56287,NaN,Tram
102608,601,771,2498,46788,NaN,Bus
234872,604,824,2473,44374,NaN,Bus
63778,600,856,2386,38830,NaN,Bus
392575,608,1061,2142,57517,NaN,Bus
324973,606,882,2100,41114,NaN,Bus
54131,551,658,1998,9647,NaN,Bus


In [155]:
t.to_csv("Tables/line_nan_delay_next.csv")

In [156]:
t = line_nan_trips.sort_values(by='departure_delay_float', ascending=False).head(20)

In [157]:
t

,route_short_name,tier_trips_count,nextbike_trips_count,trip_id,departure_delay_float,route_type_name
0,16,57,143,1900,NaN,Tram
1900,18,31,60,306,NaN,Tram
2206,516,56,35,12079,NaN,Bus
14285,537,624,1447,19316,NaN,Bus
33601,540,507,1234,12685,NaN,Bus
46286,541,9,21,3266,NaN,Bus
49552,550,197,667,4579,NaN,Bus
54131,551,658,1998,9647,NaN,Bus
63778,600,856,2386,38830,NaN,Bus
102608,601,771,2498,46788,NaN,Bus


In [158]:
t.to_csv("Tables/line_nan_delay_delay.csv")

- what is more delayed and substituted? Bus, Train, Tram?

In [34]:
type_delay = delay.groupby('route_type_name').agg({'tier_trips_count': 'sum', 'nextbike_trips_count': 'sum', 'tier_trips_end_at_station_count' : 'sum', 'nextbike_trips_end_at_station_count' : 'sum'})
#line_delay_trips = line_delay_trips.merge(delay[['route_short_name',]], on='route_short_name', how='left')
type_delay.drop_duplicates(inplace=True)
type_delay

,tier_trips_count,nextbike_trips_count,tier_trips_end_at_station_count,nextbike_trips_end_at_station_count
route_type_name,,,,
Bus,11462,30529,9402,25740
Tram,1010,2955,785,2417


In [35]:
type_no_delay = no_delay.groupby('route_type_name').agg({'tier_trips_count': 'sum', 'nextbike_trips_count': 'sum', 'tier_trips_end_at_station_count' : 'sum', 'nextbike_trips_end_at_station_count' : 'sum'})
#line_delay_trips = line_delay_trips.merge(delay[['route_short_name',]], on='route_short_name', how='left')
type_no_delay.drop_duplicates(inplace=True)
type_no_delay

,tier_trips_count,nextbike_trips_count,tier_trips_end_at_station_count,nextbike_trips_end_at_station_count
route_type_name,,,,
Bus,15225,34957,12327,29380
Tram,2955,6248,2325,5051


In [36]:
type_nan_delay = nan_delay.groupby('route_type_name').agg({'tier_trips_count': 'sum', 'nextbike_trips_count': 'sum', 'tier_trips_end_at_station_count' : 'sum', 'nextbike_trips_end_at_station_count' : 'sum'})
#line_delay_trips = line_delay_trips.merge(delay[['route_short_name',]], on='route_short_name', how='left')
type_nan_delay.drop_duplicates(inplace=True)
type_nan_delay

,tier_trips_count,nextbike_trips_count,tier_trips_end_at_station_count,nextbike_trips_end_at_station_count
route_type_name,,,,
Bus,16029,40598,12979,33955
Tram,3038,7600,2415,6240


## Service

In [37]:
# check if service in place or not
# check if service in place but delay or nan
# check if service not in place but still no delay or delay (should be empty)

In [38]:
#TO DO
# - check cancelled_trip
# - check delay category

In [39]:
# trip should have happened
# if not - unplanned 
service_in_place = data[data['service'] == 1]

In [40]:
service_in_place['departure_delay'].describe()

count                      1514773
mean     0 days 00:00:42.782365410
std      0 days 00:19:11.670259678
min                0 days 00:00:00
25%                0 days 00:00:00
50%                0 days 00:00:15
75%                0 days 00:00:30
max                1 days 00:00:00
Name: departure_delay, dtype: object

In [41]:
get_stats(service_in_place, ['tier_trips_count', 'nextbike_trips_count'])

/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)


column,tier_trips_count,nextbike_trips_count
rows,2553580,2553580
sum,50369,124636
mean,0.019725,0.048808
median,0.0,0.0
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: nextbike_trips_count, dtype: int64"
max,6,11
min,0,0
std_dev,0.156563,0.271434
variance,0.024512,0.073676
skewness,9.855624,8.111285


In [80]:
service_tier = check_micromobility_datasets('tier', service_in_place)

In [81]:
service_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.657900e+04,16579,16579,16579,16579.000000,16579.000000,11640.000000,8500.000000,16579.000000
mean,7.986886e+08,2023-07-15 12:46:09.004161792,2023-07-15 13:00:20.548887296,0 days 00:14:11.544725254,1.767257,1767.257228,1385.388832,1420.679294,851.544725
min,7.822645e+08,2023-06-30 01:45:00,2023-06-30 02:05:00,0 days 00:05:00,0.100042,100.042262,43.000000,43.000000,300.000000
25%,7.823775e+08,2023-07-07 19:35:00,2023-07-07 19:45:00,0 days 00:10:00,0.764796,764.796233,699.000000,1102.000000,600.000000
50%,8.086315e+08,2023-07-14 09:45:00,2023-07-14 09:55:00,0 days 00:10:00,1.362530,1362.529672,1136.000000,1146.000000,600.000000
75%,8.087477e+08,2023-07-22 16:50:00,2023-07-22 17:05:00,0 days 00:15:00,2.318925,2318.924552,1310.000000,1290.000000,900.000000
max,8.323141e+08,2023-08-01 01:45:00,2023-08-01 02:05:00,0 days 02:00:00,13.908704,13908.703513,9780.000000,9780.000000,7200.000000
std,1.676256e+07,NaN,NaN,0 days 00:09:51.830412191,1.474926,1474.925664,1466.436101,1493.117121,591.830412


In [82]:
#get_stats(service_tier, ['trip_duration', 'trip_length'])

In [83]:
service_next = check_micromobility_datasets('next', service_in_place)

In [84]:
service_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,4.136100e+04,41361,41361,41361,41361.000000,41361.000000,30946.000000,23511.000000,41361.000000
mean,3.793908e+07,2023-07-15 16:52:18.059041280,2023-07-15 17:07:10.285051136,0 days 00:14:52.226010009,1.740426,1740.426455,1245.547728,1281.139977,892.226010
min,3.771653e+07,2023-07-01 01:31:00,2023-07-01 01:46:00,0 days 00:05:00,0.100004,100.003661,43.000000,43.000000,300.000000
25%,3.782709e+07,2023-07-07 21:26:00,2023-07-07 21:41:00,0 days 00:10:00,0.783745,783.745359,692.000000,698.000000,600.000000
50%,3.794075e+07,2023-07-14 16:06:00,2023-07-14 16:26:00,0 days 00:15:00,1.343316,1343.315610,1143.000000,1144.000000,900.000000
75%,3.804864e+07,2023-07-22 16:06:00,2023-07-22 16:21:00,0 days 00:15:00,2.242449,2242.448614,1223.000000,1221.000000,900.000000
max,3.817174e+07,2023-08-01 03:21:00,2023-08-01 03:31:00,0 days 02:00:00,14.143293,14143.293036,9780.000000,9780.000000,7200.000000
std,1.284403e+05,NaN,NaN,0 days 00:09:51.492292666,1.447990,1447.990170,1194.754317,1268.691207,591.492293


In [85]:
#get_stats(service_next, ['trip_duration', 'trip_length'])

In [86]:
# trip should have no times
# if trip happened - data from operator not correct or different vehicle was driven 
no_service = data[data['service'] == 0]

In [87]:
no_service['departure_delay'].describe()

count                         7895
mean     0 days 01:17:10.591893603
std      0 days 05:21:29.664103546
min                0 days 00:00:00
25%                0 days 00:00:00
50%                0 days 00:00:00
75%                0 days 00:00:30
max                1 days 00:00:00
Name: departure_delay, dtype: object

In [88]:
no_service

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
50,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
51,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
52,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
53,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
54,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469624,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0,0,0,0,0,0,0,0,0,0
2469641,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0,0,0,0,0,0,0,0,0,0
2469647,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0-5,0,0,0,0,0,0,0,1,0
2469654,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0-5,0,0,0,0,0,0,0,1,0


In [89]:
get_stats(no_service, ['tier_trips_count', 'nextbike_trips_count'])

/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)


column,tier_trips_count,nextbike_trips_count
rows,11281,11281
sum,224,136
mean,0.019856,0.012056
median,0.0,0.0
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: nextbike_trips_count, dtype: int64"
max,3,3
min,0,0
std_dev,0.162415,0.123613
variance,0.026379,0.01528
skewness,9.855171,12.568949


In [90]:
no_service_tier = check_micromobility_datasets('tier', no_service)

In [91]:
no_service_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,1.280000e+02,128,128,128,128.000000,128.000000,111.000000,78.000000,128.000000
mean,8.013109e+08,2023-07-15 21:40:11.718749952,2023-07-15 21:55:37.500000,0 days 00:15:25.781250,2.237420,2237.420057,1121.891892,1465.615385,925.781250
min,7.822683e+08,2023-06-30 05:25:00,2023-06-30 05:40:00,0 days 00:05:00,0.128069,128.069115,686.000000,161.000000,300.000000
25%,7.823841e+08,2023-07-08 05:18:45,2023-07-08 05:31:15,0 days 00:10:00,0.827766,827.765856,687.000000,1106.000000,600.000000
50%,8.086559e+08,2023-07-16 00:30:00,2023-07-16 00:42:30,0 days 00:12:30,1.731083,1731.082871,1102.000000,1156.500000,750.000000
75%,8.087616e+08,2023-07-23 00:12:30,2023-07-23 00:33:45,0 days 00:20:00,2.841388,2841.387971,1137.500000,1399.250000,1200.000000
max,8.323011e+08,2023-07-31 00:20:00,2023-07-31 00:30:00,0 days 02:00:00,10.501190,10501.189675,9029.000000,8814.000000,7200.000000
std,1.753652e+07,NaN,NaN,0 days 00:13:02.510204079,1.857474,1857.474051,1092.993707,1486.134638,782.510204


In [92]:
#get_stats(no_service_tier, ['trip_duration', 'trip_length'])

In [93]:
no_service_next = check_micromobility_datasets('next', no_service)

In [94]:
no_service_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,9.500000e+01,95,95,95,95.000000,95.000000,76.000000,54.000000,95.000000
mean,3.794873e+07,2023-07-17 22:44:03.157894400,2023-07-17 23:00:12.631578880,0 days 00:16:09.473684210,2.369974,2369.974241,1366.184211,1531.333333,969.473684
min,3.773277e+07,2023-07-01 06:11:00,2023-07-01 06:36:00,0 days 00:05:00,0.102409,102.408806,161.000000,43.000000,300.000000
25%,3.782785e+07,2023-07-09 00:33:30,2023-07-09 00:51:00,0 days 00:10:00,0.980179,980.178710,687.000000,1110.000000,600.000000
50%,3.795270e+07,2023-07-21 01:41:00,2023-07-21 02:01:00,0 days 00:15:00,1.806297,1806.297178,1124.500000,1162.000000,900.000000
75%,3.805482e+07,2023-07-23 01:36:00,2023-07-23 01:56:00,0 days 00:20:00,3.253313,3253.313148,1500.000000,1431.000000,1200.000000
max,3.816733e+07,2023-07-30 03:01:00,2023-07-30 03:26:00,0 days 00:45:00,8.762867,8762.866509,8507.000000,8814.000000,2700.000000
std,1.282129e+05,NaN,NaN,0 days 00:08:17.883538507,1.964924,1964.923528,1529.312531,1765.030343,497.883539


In [95]:
#get_stats(no_service_next, ['trip_duration', 'trip_length'])

In [96]:
no_trip_no_service = no_service[no_service['actual_departure_time'].isna()]
no_trip_no_service

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
58,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,early morning,8,1,0,0,-1,0
505,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,151,313-551-008-2071.2.22:013200-34-157_9BC2DD26-8...,Bonn Hbf,1,...,0,0,0,0,0,2,0,0,-1,0
793,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,151,314-551-008-2071.2.22:023200-34-157_3D30A6B3-5...,Bonn Hbf,1,...,0,0,0,early morning,8,2,0,0,-1,0
1037,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,151,320-551-008-2071.2.22:033200-34-157_85CDB463-9...,Bonn Hbf,1,...,0,0,1,0,0,2,0,0,-1,1
1293,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,151,321-551-008-2071.2.22:043200-34-157_7EC0AE0C-A...,Bonn Hbf,1,...,0,0,0,early morning,8,2,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469347,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651303-65-006-1593.1.12:240400-24-179_0D61D562...,Kopenhagener Str.,0,...,0,0,0,0,0,0,0,0,-1,1
2469364,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651303-65-006-1593.1.12:240400-24-179_0D61D562...,Kopenhagener Str.,0,...,0,0,0,0,0,0,0,0,-1,1
2469370,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651326-65-006-1406.1.12:241400-15-179_E7BE51A3...,Beuel Bf,1,...,0,0,0,0,0,0,0,0,-1,0
2469512,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651305-65-006-1593.1.12:243600-10-179_EF0CBDA0...,Beuel Bf,0,...,0,0,0,0,0,0,0,0,-1,0


In [97]:
get_stats(no_trip_no_service, ['tier_trips_count', 'nextbike_trips_count'])

/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)


column,tier_trips_count,nextbike_trips_count
rows,3386,3386
sum,100,53
mean,0.029533,0.015653
median,0.0,0.0
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: nextbike_trips_count, dtype: int64"
max,3,3
min,0,0
std_dev,0.208424,0.153896
variance,0.043441,0.023684
skewness,8.713174,12.750556


In [98]:
short = no_trip_no_service[['scheduled_arrival_time', 'actual_arrival_time','actual_departure_time','scheduled_departure_time', 'arrival_delay', 'arrival_delay_float', 'service', 'cancelled_trip', 'delay_category' ]]
short

,scheduled_arrival_time,actual_arrival_time,actual_departure_time,scheduled_departure_time,arrival_delay,arrival_delay_float,service,cancelled_trip,delay_category
58,2023-06-30 05:20:00,2023-07-01 05:20:00,NaT,2023-06-30 05:20:00,1 days,0.0,0,0,-1
505,2023-07-01 02:20:00,2023-07-02 02:20:00,NaT,2023-07-01 02:20:00,1 days,0.0,0,0,-1
793,2023-07-01 03:20:00,2023-07-02 03:20:00,NaT,2023-07-01 03:20:00,1 days,0.0,0,0,-1
1037,2023-07-01 04:20:00,NaT,NaT,2023-07-01 04:20:00,NaT,NaN,0,1,-1
1293,2023-07-01 05:20:00,2023-07-02 05:20:00,NaT,2023-07-01 05:20:00,1 days,0.0,0,0,-1
...,...,...,...,...,...,...,...,...,...
2469347,2023-07-31 00:32:00,NaT,NaT,2023-07-31 00:32:00,NaT,NaN,0,1,-1
2469364,2023-07-31 00:34:00,NaT,NaT,2023-07-31 00:34:00,NaT,NaN,0,1,-1
2469370,2023-07-31 00:34:00,2023-07-31 00:33:40,NaT,2023-07-31 00:34:00,0 days,0.0,0,0,-1
2469512,2023-07-31 00:46:00,2023-07-31 00:45:40,NaT,2023-07-31 00:46:00,0 days,0.0,0,0,-1


In [99]:
no_trip_no_service_tier = check_micromobility_datasets('tier', no_trip_no_service)

In [100]:
no_trip_no_service_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,5.800000e+01,58,58,58,58.000000,58.000000,52.000000,35.000000,58.000000
mean,8.027731e+08,2023-07-16 16:52:04.137931520,2023-07-16 17:05:36.206896640,0 days 00:13:32.068965517,2.132035,2132.034789,916.653846,1577.714286,812.068966
min,7.822688e+08,2023-06-30 05:25:00,2023-06-30 05:40:00,0 days 00:05:00,0.128069,128.069115,686.000000,161.000000,300.000000
25%,7.823692e+08,2023-07-09 00:15:00,2023-07-09 00:26:15,0 days 00:10:00,0.825748,825.747852,687.000000,1109.000000,600.000000
50%,8.086624e+08,2023-07-16 00:22:30,2023-07-16 00:35:00,0 days 00:10:00,1.547808,1547.808382,687.000000,1140.000000,600.000000
75%,8.087865e+08,2023-07-23 01:28:45,2023-07-23 01:35:00,0 days 00:15:00,2.548988,2548.988112,1115.000000,1308.500000,900.000000
max,8.323011e+08,2023-07-31 00:20:00,2023-07-31 00:30:00,0 days 00:55:00,8.079087,8079.087067,2516.000000,8814.000000,3300.000000
std,1.815021e+07,NaN,NaN,0 days 00:08:31.988756115,1.866999,1866.999007,375.727941,1812.181551,511.988756


In [101]:
#get_stats(no_trip_no_service_tier, ['trip_duration', 'trip_length'])

In [102]:
no_trip_no_service_next = check_micromobility_datasets('next', no_trip_no_service)

In [103]:
no_trip_no_service_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,3.300000e+01,33,33,33,33.000000,33.000000,26.000000,18.000000,33.000000
mean,3.792300e+07,2023-07-18 04:22:40.000000256,2023-07-18 04:40:23.636363520,0 days 00:17:43.636363636,2.611653,2611.653056,912.961538,1510.833333,1063.636364
min,3.773277e+07,2023-07-02 00:21:00,2023-07-02 00:36:00,0 days 00:05:00,0.111373,111.372953,161.000000,161.000000,300.000000
25%,3.782145e+07,2023-07-09 01:41:00,2023-07-09 02:21:00,0 days 00:10:00,1.181620,1181.620092,687.000000,1116.500000,600.000000
50%,3.793241e+07,2023-07-22 00:26:00,2023-07-22 00:41:00,0 days 00:15:00,1.790664,1790.664056,687.000000,1173.500000,900.000000
75%,3.797651e+07,2023-07-23 01:36:00,2023-07-23 01:46:00,0 days 00:20:00,3.140836,3140.836425,1112.750000,1221.000000,1200.000000
max,3.812858e+07,2023-07-30 01:46:00,2023-07-30 01:56:00,0 days 00:45:00,8.762867,8762.866509,2667.000000,8814.000000,2700.000000
std,1.237910e+05,NaN,NaN,0 days 00:09:41.387447092,2.231655,2231.654930,470.038763,1852.968945,581.387447


In [104]:
#get_stats(no_trip_no_service_next, ['trip_duration', 'trip_length'])

In [105]:
trip_no_service = no_service[no_service['actual_departure_time'].notnull()]
trip_no_service

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_span,arrival_delay_span,weather_int,time_span_arrival,time_span_int_arrival,weekend,time_span,time_span_int,delay_category,cancelled_trip
50,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
51,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
52,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
53,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
54,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,130,256-551-008-2071.2.22:043200-34-157_5DA786E0-4...,Bonn Hbf,1,...,0,0,1,0,0,1,early morning,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469598,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0-5,0,0,0,0,0,0,0,1,0
2469624,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0,0,0,0,0,0,0,0,0,0
2469641,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0,0,0,0,0,0,0,0,0,0
2469647,65,6,65,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,141,651328-65-006-1406.1.12:244400-15-179_4B6B4075...,Beuel Bf,1,...,0-5,0,0,0,0,0,0,0,1,0


In [106]:
get_stats(trip_no_service, ['tier_trips_count', 'nextbike_trips_count'])

/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:51: RuntimeWarning: invalid value encountered in scalar divide
  qd = iqr / (percentile_75 + percentile_25)
/tmp/ipykernel_384496/1947819274.py:55: RuntimeWarning: invalid value encountered in scalar divide
  cvqd = iqr / (percentile_75 + percentile_25)


column,tier_trips_count,nextbike_trips_count
rows,7895,7895
sum,124,83
mean,0.015706,0.010513
median,0.0,0.0
mode,"0 0 Name: tier_trips_count, dtype: int64","0 0 Name: nextbike_trips_count, dtype: int64"
max,3,2
min,0,0
std_dev,0.137871,0.10803
variance,0.019008,0.011671
skewness,10.003518,11.063798


In [107]:
short = trip_no_service[['scheduled_arrival_time', 'actual_arrival_time','actual_departure_time','scheduled_departure_time', 'arrival_delay', 'arrival_delay_float', 'service', 'cancelled_trip', 'delay_category' ]]
short

,scheduled_arrival_time,actual_arrival_time,actual_departure_time,scheduled_departure_time,arrival_delay,arrival_delay_float,service,cancelled_trip,delay_category
50,2023-06-30 05:04:00,NaT,2023-07-01 05:04:00,2023-06-30 05:04:00,NaT,NaN,0,0,0
51,2023-06-30 05:06:00,NaT,2023-07-01 05:06:00,2023-06-30 05:06:00,NaT,NaN,0,0,0
52,2023-06-30 05:08:00,NaT,2023-07-01 05:08:00,2023-06-30 05:08:00,NaT,NaN,0,0,0
53,2023-06-30 05:10:00,NaT,2023-07-01 05:10:00,2023-06-30 05:10:00,NaT,NaN,0,0,0
54,2023-06-30 05:13:00,NaT,2023-07-01 05:13:00,2023-06-30 05:13:00,NaT,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...
2469598,2023-07-31 00:54:00,2023-07-31 00:54:30,2023-07-31 00:54:50,2023-07-31 00:54:00,0 days 00:00:30,0.500000,0,0,1
2469624,2023-07-31 00:57:00,2023-07-31 00:56:20,2023-07-31 00:57:00,2023-07-31 00:57:00,0 days 00:00:00,0.000000,0,0,0
2469641,2023-07-31 01:00:00,2023-07-31 00:59:40,2023-07-31 01:00:00,2023-07-31 01:00:00,0 days 00:00:00,0.000000,0,0,0
2469647,2023-07-31 01:01:00,2023-07-31 01:00:50,2023-07-31 01:01:10,2023-07-31 01:01:00,0 days 00:00:00,0.000000,0,0,1


In [108]:
trip_no_service_tier = check_micromobility_datasets('tier', trip_no_service)

In [109]:
trip_no_service_tier.describe()

,tier_trips_id,tier_trips_start_time,tier_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,9.700000e+01,97,97,97,97.000000,97.000000,84.000000,60.000000,97.000000
mean,8.010218e+08,2023-07-15 14:02:16.082473984,2023-07-15 14:18:21.030927872,0 days 00:16:04.948453608,2.262589,2262.588921,1183.142857,1307.283333,964.948454
min,7.822683e+08,2023-07-01 01:45:00,2023-07-01 01:50:00,0 days 00:05:00,0.128069,128.069115,686.000000,683.000000,300.000000
25%,7.823888e+08,2023-07-09 00:10:00,2023-07-09 00:25:00,0 days 00:10:00,0.882796,882.796018,687.000000,1106.000000,600.000000
50%,8.086550e+08,2023-07-16 00:20:00,2023-07-16 00:30:00,0 days 00:15:00,1.925899,1925.898869,1102.500000,1156.500000,900.000000
75%,8.087580e+08,2023-07-22 03:50:00,2023-07-22 04:05:00,0 days 00:20:00,3.192307,3192.307383,1139.500000,1411.500000,1200.000000
max,8.323011e+08,2023-07-30 02:20:00,2023-07-30 02:40:00,0 days 02:00:00,10.501190,10501.189675,9029.000000,8437.000000,7200.000000
std,1.707316e+07,NaN,NaN,0 days 00:14:18.661899546,1.778128,1778.127747,1230.345237,1007.292059,858.661900


In [110]:
#get_stats(trip_no_service_tier, ['trip_duration', 'trip_length'])

In [111]:
trip_no_service_next = check_micromobility_datasets('next', trip_no_service)

In [112]:
trip_no_service_next.describe()

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,end_stop_id,trip_duration_seconds
count,7.000000e+01,70,70,70,70.000000,70.000000,58.000000,44.000000,70.000000
mean,3.796011e+07,2023-07-17 11:02:04.285714432,2023-07-17 11:17:55.714285568,0 days 00:15:51.428571428,2.340965,2340.964804,1531.206897,1639.818182,951.428571
min,3.774192e+07,2023-07-01 06:11:00,2023-07-01 06:36:00,0 days 00:05:00,0.102409,102.408806,161.000000,43.000000,300.000000
25%,3.784584e+07,2023-07-09 00:21:00,2023-07-09 00:42:15,0 days 00:10:00,0.973803,973.802826,791.500000,1113.750000,600.000000
50%,3.798651e+07,2023-07-16 02:03:30,2023-07-16 02:16:00,0 days 00:15:00,1.813033,1813.033054,1168.000000,1167.500000,900.000000
75%,3.806813e+07,2023-07-23 01:34:45,2023-07-23 02:02:15,0 days 00:20:00,3.272290,3272.289610,1500.000000,1500.250000,1200.000000
max,3.816733e+07,2023-07-30 03:01:00,2023-07-30 03:26:00,0 days 00:35:00,8.704533,8704.533068,8507.000000,8814.000000,2100.000000
std,1.277591e+05,NaN,NaN,0 days 00:07:28.101585643,1.913857,1913.856909,1711.907541,1939.035282,448.101586


In [113]:
#get_stats(trip_no_service_next, ['trip_duration', 'trip_length'])

In [114]:
service = data.groupby('service').agg({'tier_trips_count': 'sum', 'nextbike_trips_count': 'sum', 'tier_trips_end_at_station_count' : 'sum', 'nextbike_trips_end_at_station_count' : 'sum'})
#line_delay_trips = line_delay_trips.merge(delay[['route_short_name',]], on='route_short_name', how='left')
service.drop_duplicates(inplace=True)
service

,tier_trips_count,nextbike_trips_count,tier_trips_end_at_station_count,nextbike_trips_end_at_station_count
service,,,,
0,224,136,200,112
1,50369,124636,40725,104292


In [115]:
data['service'].sum()

2553580

In [116]:
service = data.groupby('service').agg({'cancelled_trip': 'sum', 'tier_trips_count': 'sum','nextbike_trips_count': 'sum', 'tier_trips_end_at_station_count' : 'sum', 'nextbike_trips_end_at_station_count' : 'sum'})
#line_delay_trips = line_delay_trips.merge(delay[['route_short_name',]], on='route_short_name', how='left')
service.drop_duplicates(inplace=True)
service

,cancelled_trip,tier_trips_count,nextbike_trips_count,tier_trips_end_at_station_count,nextbike_trips_end_at_station_count
service,,,,,
0,3017,224,136,200,112
1,1002596,50369,124636,40725,104292


In [117]:
data['cancelled_trip'].sum()

1005613

In [160]:
service = delay.groupby('service').agg({'cancelled_trip': 'sum', 'tier_trips_count': 'sum','nextbike_trips_count': 'sum', 'tier_trips_end_at_station_count' : 'sum', 'nextbike_trips_end_at_station_count' : 'sum'})
#line_delay_trips = line_delay_trips.merge(delay[['route_short_name',]], on='route_short_name', how='left')
service.drop_duplicates(inplace=True)
service

,cancelled_trip,tier_trips_count,nextbike_trips_count,tier_trips_end_at_station_count,nextbike_trips_end_at_station_count
service,,,,,
0,0,35,37,34,32
1,0,12437,33447,10153,28125


In [161]:
service = nan_delay.groupby('service').agg({'cancelled_trip': 'sum', 'tier_trips_count': 'sum','nextbike_trips_count': 'sum', 'tier_trips_end_at_station_count' : 'sum', 'nextbike_trips_end_at_station_count' : 'sum'})
#line_delay_trips = line_delay_trips.merge(delay[['route_short_name',]], on='route_short_name', how='left')
service.drop_duplicates(inplace=True)
service

,cancelled_trip,tier_trips_count,nextbike_trips_count,tier_trips_end_at_station_count,nextbike_trips_end_at_station_count
service,,,,,
0,3017,60,28,55,25
1,1002596,19007,48170,15339,40170


In [162]:
service.to_csv("Tables/Service_Nan.csv")